# Privacy Preserving Technologies
The first two exercises concerns an example situation where course staff require students to disclose whether they have cheated on the final exam of the course. Naturally there is a conflict of interest here, students don't want their answers to be personally identifiable for fear of repercussions, and course staff want useful aggregate statistics.

## 1) Differential Privacy
As a first solution the students suggest to use differential privacy techniques to address the problem. In this first part we build a toy dataset of student answers and implement central and local differential privacy methods. Finally, you will reflect on what methods and parameters you would prefer to have from the student perspective. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tenseal as ts
import secrets
import hashlib
from sudoku import Sudoku

In [ ]:
N = 50
TRUE_PROBABILITY = 0.20 
PRIVACY_PARAMETER = 0.5 # In [0, 1]

np.random.seed(42)

def generate_data(population, prob):
    """Generates the ground truth dataset (1 = Yes, 0 = No)"""
    dataset = [1] * int(prob*population) + [0] * int((1-prob) * population)
    np.random.shuffle(dataset)
    return dataset

In [ ]:
def laplace(dataset, sensitivity, epsilon):
    """Add Laplace noise to the true count"""
    true_count = np.sum(dataset)
    
    # TODO determine scale and add noise
    
    # ENDTODO
    
    private_count = true_count + noise
    return int(private_count)

In [ ]:
def randomized_response(dataset):
    """
    Simulate the local 'Coin Flip' mechanism for differential privacy.
    Algorithm:
    For each answer in the dataset:
    - Flip Coin 1.
    - If Heads (PRIVACY_PARAMETER%): 
        - Answer Truthfully.
    - If Tails (1-PRIVACY_PARAMETER%): 
        - Flip Coin 2.
        - If Heads (50%): Answer 'Yes' (1).
        - If Tails (50%): Answer 'No' (0).
    """
    # TODO implement randomized response
    responses = ...
    # ENDTODO
    return responses

def debias_randomized_response(noisy_counts, population):
    """
    Reverse engineer the noise to estimate the true count.
    
    Math derivation:
    P(Yes_Reported) = P(Heads)*P(True_Yes) + P(Tails)*P(Random_Yes)
    """
    # TODO calculate the debiasing based on the randomized response implementation
    
    # ENDTODO
    return int(estimated_true_prob * population)

In [ ]:
# Construct the dataset
data = generate_data(N, TRUE_PROBABILITY)
true_count = np.sum(data)
print(f"Direct Approach:")
print(f"True Count: {true_count}")

In [ ]:
# TODO determine sensitivity:
# https://en.wikipedia.org/wiki/Additive_noise_differential_privacy_mechanisms
central_estimate = laplace(data, ..., PRIVACY_PARAMETER)
# ENDTODO

central_error = abs(central_estimate - true_count)

print(f"Central Approach: Laplace, e={PRIVACY_PARAMETER}")
print(f"Laplace Count: {central_estimate:d}")
print(f"Error: {central_error:d} ({(central_error/N)*100:.2f}% of N)")

In [ ]:
# Simulate local randomized response
local_noisy_responses = randomized_response(data)
local_observed_yes = np.sum(local_noisy_responses)

# We must debias the local results to get a useful number
local_estimate = debias_randomized_response(local_observed_yes, N)
local_error = abs(local_estimate - true_count)

print(f"Local Approach: Randomized Response, coin 1 bias = {PRIVACY_PARAMETER}")
print(f"Noisy Count: {local_observed_yes}")
print(f"Debiased Estimate: {local_estimate}")
print(f"Error: {local_error} ({(local_error/N)*100:.2f}% of N)")

Now you have implemented both the central and local differential privacy methods. In this last part you will determine from the perspective of a student which method and what parameters you would prefer to use. Think about and answer the following questions (Possibly using experimentation to inform your answers) 
1. Whether you prefer the central or the local method, and why?
2. Assuming we have a class of 200 students (i.e. N = 200), which PRIVACY_PARAMETER would you be comfortable with? What kind of error can the course staff expect for your chosen PRIVACY_PARAMETER?
3. In the hypothetical that we could increase N by e.g. including other classes in the response, would you prefer it for your privacy? Does increasing N have an effect on the error the course staff should expect? 


## 2) Homomorphic Encryption
The course staff deemed the error of the differential privacy methods above too high to be useful, so they reject the student's proposal. Instead the students now suggest a voting system using Homomorphic Encryption as a technique to achieve privacy, which allows the course staff to get accurate aggregate statistics. You will implement the functionality of the teller and the vote casting.

In [ ]:
class ElectionAuthority:
    def __init__(self):
        self._context = ts.context(
            ts.SCHEME_TYPE.BFV, 
            poly_modulus_degree=4096, 
            plain_modulus=1032193
        )
        self._context.generate_galois_keys()
        self._context.generate_relin_keys()

    def get_public_context(self):
        return ts.context_from(self._context.serialize(save_secret_key=False))

    def decrypt_tally(self, encrypted_tally):
        return encrypted_tally.decrypt(secret_key=self._context.secret_key())[0]

In [ ]:
class Teller:
    def __init__(self):
        pass

    def tally_votes(self, votes):
        # TODO implement homomorphic vote tallying
        tally = ...
        
        # ENDTODO
        return tally

In [ ]:
class Voter:
    def __init__(self, ctx, p):
        # ctx is the *public* encryption context, i.e. it only contains the public key for encrypting towards the EncryptionAuthority
        self.ctx = ctx
        # This voter has a *private* opinion based on which they vote
        self.__opinion = np.random.choice([1, 0], p=[p, 1-p])

    def vote(self):
        # TODO implement casting an encrypted vote
        vote = ...
        # ENDTODO
        return vote

In [ ]:
# Initialize
EA = ElectionAuthority()
teller = Teller()
public_ctx = EA.get_public_context()
voters = [Voter(public_ctx, TRUE_PROBABILITY) for _ in range(N)]
votes = [voter.vote() for voter in voters]

Above we have initialized all the relevant parties for the voting procedure. Take the role of an attacker and try to figure out if you can accomplish any of the following:
- Can you figure out the opinion of a single voter based on the complete list of votes? (using only the `votes` and `public_ctx` which are publicly available)
- Can you figure out the opinion of a single voter by repeatedly requesting them to vote? (using only the `votes`, `public_ctx`, and `Voter.vote()` method)

In [ ]:
encrypted_tally = teller.tally_votes(votes)
print(f"Tally Complete. Result is (snippet):")
print(f"{encrypted_tally.serialize()[-200:]}")

In [ ]:
# Election Authority gets the encrypted tally and reveals the final result
final_tally = EA.decrypt_tally(encrypted_tally)
print(f"Tally revealed by Election Authority: {final_tally}")
print(f"Actual (secret) tally: {sum([v._Voter__opinion for v in voters])}")

Reflect on the voting scheme above and identify who should hold which role in the classroom:
- Who are the voters?
- Who should be the Teller?
- Who should be the Election Authority?
- Should the teller and the election authority be the same person in this protocol? Why or why not?

## 3) Zero Knowledge Proofs

In this final part of the lab we will implement an example of a zero knowledge proof based on Sudokus. Suppose that you and a friend have a habit of solving challenging Sudokus every day. Your friend does not believe that you have solved today's Sudoku, and you would like to prove to them that you did but you don't want to reveal the answer. Naturally, you devise a zero-knowledge proof:

- The protocol consists of two stages:
    - Commit stage
    1. You create a random mapping of original digits to new digits and apply it to your solution
    2. You 'commit' to your solution by publishing the hashes (value + salt) of your remapped solution
    - Verification stage
    1. The verifier can perform a challenge on the row, column, or subgrid of the remapped solution, or on the consistency with the public puzzle. The prover provides the values of the requested row, column, or subgrid without any mapping information, or with the specific values of the publicly known puzzle spaces along with the mapping.
    2. The verifier confirms that the challenge response conforms with the commitment before, and then checks for correctness. 

The verifier and prover will go through multiple rounds until the verifier is satisfied that the prover knows the solution. The full implementation of the prover is provided to you, your task is to implement both the Verifier and a CheatingProver. And finally you will determine what parameters should be used for your Sudoku-ZKP.

In [ ]:
class Commitment:
    def __init__(self, value):
        self.value = int(value) 
        self.salt = secrets.token_hex(16)
        self.hash = self._calculate_hash(self.value, self.salt)

    @staticmethod
    def _calculate_hash(value, salt):
        data = f"{value}{salt}".encode()
        return hashlib.sha256(data).hexdigest()
    
    def reveal(self):
        return self.value, self.salt

    def verify(self, value, salt):
        return Commitment._calculate_hash(value, salt) == self.hash

    @staticmethod
    def verify_hash(value, salt, expected_hash):
        return Commitment._calculate_hash(value, salt) == expected_hash

In [ ]:
class SudokuProver:
    def __init__(self, solution, puzzle):
        self.solution = np.array(solution.board) 
        self.puzzle = np.array(puzzle.board)
        self.puzzle_indexes = np.where(self.puzzle != None)
        self.n = self.solution.shape[0]
        self.box_size = int(np.sqrt(self.n))

    def set_commitment(self, grid):
        commitments = []
        hashes = []
        
        for r in range(self.n):
            c_row = []
            h_row = []
            for c in range(self.n):
                val = grid[r, c]
                comm = Commitment(val)
                c_row.append(comm)
                h_row.append(comm.hash)
            commitments.append(c_row)
            hashes.append(h_row)
            
        self.current_commitments = np.array(commitments, dtype=object)
        return np.array(hashes)
        
    def generate_proof_round(self):
        """Permute values and publish commitment hashes"""
        digits = np.arange(1, self.n + 1)
        shuffled = np.copy(digits)
        np.random.shuffle(shuffled)
        
        mapping_array = np.zeros(self.n + 1, dtype=int)
        mapping_array[digits] = shuffled
        
        permuted_values = mapping_array[self.solution]
        # this is the mapping back from shuffled to original, to be released only on consistency challenge
        self.last_mapping = dict(zip(shuffled, digits))
        return self.set_commitment(permuted_values)

    def respond_to_challenge(self, challenge_type, index):
        """Respond to given challenge of the verifier"""
        commits = self.current_commitments
        response = []
        mapping = None
        
        if challenge_type == 'row':
            response = [c.reveal() for c in commits[index, :]]
        elif challenge_type == 'col':
            response = [c.reveal() for c in commits[:, index]]
        elif challenge_type == 'subgrid':
            r_start = (index // self.box_size) * self.box_size
            c_start = (index % self.box_size) * self.box_size
            
            subgrid_objs = commits[r_start : r_start + self.box_size, 
                                   c_start : c_start + self.box_size]

            response = [c.reveal() for c in subgrid_objs.flatten()]
        elif challenge_type == 'consistency':
            response = [c.reveal() for c in commits[self.puzzle_indexes]]
            mapping = self.last_mapping
        return response, mapping

In [ ]:
class Verifier:
    def __init__(self, rounds, puzzle):
        self.rounds = rounds
        self.puzzle = np.array(puzzle.board)
        self.puzzle_indexes = np.where(self.puzzle != None)
        self.puzzle_vals = self.puzzle[self.puzzle_indexes]
        self.n = self.puzzle.shape[0]
        self.box_size = int(np.sqrt(self.n))
        self.expected_vals = list(range(1, self.n + 1))
        self.challenge_types = ['row', 'col', 'subgrid', 'consistency']

    def generate_challenge(self):
        return np.random.choice(self.challenge_types), np.random.randint(0, self.n - 1)
        
    def verify(self, prover, v = True):
        """
        Verify that the given :prover: knows the solution to self.puzzle
        Returns the tuple (verified_bool, round_nr) where
        :verified_bool: indicates if the prover successfully passed verification
        :round_nr: indicates at which round the verification exited (i.e. self.rounds-1 for successful verification)
        """
        for i in range(self.rounds):
            commitment_grid = prover.generate_proof_round()
            c_type, c_index = self.generate_challenge()
            revealed_data, revealed_mapping = prover.respond_to_challenge(c_type, c_index)

            # TODO implement the verification of the challenge
            if c_type == 'row':
                challenged_hashes = commitment_grid[c_index, :]
            # Get the challenged hashes for the other challenge types


            # Verify that the hashes are correct
            for (val, salt), expected_hash in zip(revealed_data, challenged_hashes):
                ...
        
            if c_type == 'consistency':
                # check that the values with the mapping are correct for the known puzzle
                ...
            else:
                # check that the values conform to the expected set
                ...
                
            # ENDTODO
        
        if v:
            print(f"Verification successful using {self.rounds} rounds.")
        return True, i

In [ ]:
class CheatingProver(SudokuProver):
    def generate_proof_round(self):
        # TODO implement a smarter cheater
        # 1:1 mapping
        self.last_mapping = dict(zip(list(range(1, self.n+1)), list(range(1, self.n+1))))
        
        # Create a fake grid where every row is 1..N
        fake_grid = np.tile(np.arange(1, self.n+1), (self.n, 1))
        # ENDTODO
        return self.set_commitment(fake_grid)

In [ ]:
rounds = 20
box_size = 3

puzzle = Sudoku(box_size, seed=42, difficulty=1)
prover = SudokuProver(puzzle.solve(), puzzle)
cheater = CheatingProver(puzzle, puzzle)
verifier = Verifier(rounds, puzzle)

Complete the implementations above to get a complete verifier, which you can test on the real prover and your cheating prover. Try to create a cheating prover which passes rounds with the highest probability (obviously without actually solving the sudoku).

Implement some parts, try to improve the cheater (without solving puzzle obviously), how many rounds can you make it into the protocol? What is the (empirical) chance that your cheating prover passes the first round? With how many rounds would you be 'certain' that somebody is not cheating?

In [ ]:
# Test your verifier on a real prover, this should result in (True, rounds-1)
verifier.verify(prover, True)

In [ ]:
# Test your verifier on your cheating prover, this should result in (False, *)
verifier.verify(cheater, True)

With your finished implementations, estimate:
- The probability that your cheating prover passes a challenge round.
- The average number of rounds that your cheating prover successfully passes.
- The number of rounds required by the protocol to be 'certain' that the prover really knows the solution and is not cheating.

Finally, reflect on the protocol. Do you think it is a good zero-knowledge proof? Does it satisfy all properties?